<a href="https://colab.research.google.com/github/FrancisOyebanji/Hello-World/blob/master/Andrei_Transformer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
#saved
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-04-30 08:38:51--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2023-04-30 08:38:51 (54.1 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [27]:
with open('input.txt', 'r',encoding = 'utf-8')as f:
  text = f.read()

In [28]:
print('lenght of dataset in characters:', len(text))

lenght of dataset in characters: 1115394


In [29]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [30]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [31]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [32]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [33]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [34]:
block_size = 8
train_data[:block_size+1]
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  taget = y[t]

Due to data complexities, chunks of data instead of the entire data set is used to train the data intermitently

In [35]:
torch.manual_seed(1336)
batch_size = 4 # try different amount of batch size and see if it changes run time or performance
block_size = 8


def get_batch(split):
  # generate a small batch of data of input x and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1: i + block_size + 1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('input:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)


print('-------')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f'when input is {context.tolist()} the target: {target}')


input:
torch.Size([4, 8])
tensor([[39, 59, 57, 43,  8,  0,  0, 15],
        [ 5, 42,  1, 17, 50, 47, 64, 39],
        [52, 43,  5, 43, 56,  1, 57, 46],
        [ 1, 42, 43, 57, 41, 43, 52, 58]])
targets:
torch.Size([4, 8])
tensor([[59, 57, 43,  8,  0,  0, 15, 24],
        [42,  1, 17, 50, 47, 64, 39, 40],
        [43,  5, 43, 56,  1, 57, 46, 39],
        [42, 43, 57, 41, 43, 52, 58,  8]])
-------
when input is [39] the target: 59
when input is [39, 59] the target: 57
when input is [39, 59, 57] the target: 43
when input is [39, 59, 57, 43] the target: 8
when input is [39, 59, 57, 43, 8] the target: 0
when input is [39, 59, 57, 43, 8, 0] the target: 0
when input is [39, 59, 57, 43, 8, 0, 0] the target: 15
when input is [39, 59, 57, 43, 8, 0, 0, 15] the target: 24
when input is [5] the target: 42
when input is [5, 42] the target: 1
when input is [5, 42, 1] the target: 17
when input is [5, 42, 1, 17] the target: 50
when input is [5, 42, 1, 17, 50] the target: 47
when input is [5, 42, 1, 17

In [36]:
print(xb)

tensor([[39, 59, 57, 43,  8,  0,  0, 15],
        [ 5, 42,  1, 17, 50, 47, 64, 39],
        [52, 43,  5, 43, 56,  1, 57, 46],
        [ 1, 42, 43, 57, 41, 43, 52, 58]])


In [45]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
  
  def forward(self,idx, targets):
    # idx and targers are both (B,T) tensor of integers
    # B = batch (batch_size), T = time (block_size), C = Channel(vocab_size)
    logits = self.token_embedding_table(idx)

    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = targets.view(-1)
    loss = F.cross_entropy(logits, targets)

    return logits,loss

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(out.shape)
print(loss)

torch.Size([4, 8, 65])
tensor(4.5520, grad_fn=<NllLossBackward0>)
